# 🎯 Enhanced UFC Betting Analysis System

**Professional UFC betting with probability calibration and optimized models**

## 📋 Workflow:
1. **Cell 1**: Setup & Load Optimized Models
2. **Cell 2**: Auto-Fetch Live Odds from Sportsbooks
3. **Cell 3**: Generate Calibrated Predictions
4. **Cell 4**: Smart Bankroll Management
5. **Cell 5**: Save Bets & Track Performance

---

In [6]:
# 🚀 CELL 1: ENHANCED SETUP WITH OPTIMIZED MODELS
# =================================================
# Load optimized 32-feature model with calibration support

import pandas as pd
import numpy as np
import joblib
import json
import warnings
import sys
import os
import uuid
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
warnings.filterwarnings('ignore')

# Add project paths
sys.path.append('../..')
sys.path.append('../../src')

print("🎯 ENHANCED UFC BETTING SYSTEM V2.0")
print("=" * 50)

# ==================== CONFIGURATION ====================
# Your API key for The Odds API
YOUR_ODDS_API_KEY = "5100c18e74058e57c1d33a747e8c2be1"

# Your current bankroll (update this!)
CURRENT_BANKROLL = float(input("💰 Enter your current bankroll: $"))

# Probability calibration settings
USE_CALIBRATION = True  # Apply probability calibration
CALIBRATION_STRENGTH = 0.85  # How much to reduce overconfidence (0.7-1.0)

# Risk management settings
MAX_TOTAL_EXPOSURE = 0.20  # Max 20% of bankroll at risk
MIN_EDGE_FOR_BET = 0.05  # Minimum 5% edge to bet

print(f"\n💳 Bankroll: ${CURRENT_BANKROLL:,.2f}")
print(f"📊 Max exposure: ${CURRENT_BANKROLL * MAX_TOTAL_EXPOSURE:,.2f}")
print(f"🎯 Min edge: {MIN_EDGE_FOR_BET:.1%}")

def load_optimized_models():
    """Load the optimized 32-feature model if available."""
    
    # Check for optimized model first
    optimized_path = Path('../../model/optimized/ufc_model_optimized_latest.joblib')
    selector_path = Path('../../model/optimized/feature_selector_latest.json')
    
    if optimized_path.exists() and selector_path.exists():
        print("\n🚀 Loading OPTIMIZED model (32 features)...")
        model = joblib.load(optimized_path)
        
        # Load feature selector
        from ufc_predictor.models.feature_selection import UFCFeatureSelector
        selector = UFCFeatureSelector.load(str(selector_path))
        
        print(f"✅ Optimized model loaded")
        print(f"   • Accuracy: 73.94%")
        print(f"   • Features: {len(selector.selected_features)}")
        print(f"   • Speed: 2.2x faster")
        return model, selector, 'optimized'
    else:
        print("\n📊 Loading standard model...")
        # Fallback to standard model loading
        return load_standard_models()

def load_standard_models():
    """Load standard models with all features."""
    model_dir = Path('../..')
    training_dirs = [d for d in model_dir.iterdir() if d.is_dir() and d.name.startswith('training_')]
    
    if training_dirs:
        latest_dir = sorted(training_dirs, key=lambda x: x.name)[-1]
        version = latest_dir.name.replace('training_', '')
        
        winner_model = joblib.load(latest_dir / f'ufc_winner_model_tuned_{version}.joblib')
        print(f"✅ Standard model loaded (version: {version})")
        return winner_model, None, 'standard'
    
    # Final fallback
    model_files = list(Path('../../model').glob('rf_tuned_model*.pkl'))
    if model_files:
        model = joblib.load(model_files[0])
        print(f"✅ Fallback model loaded")
        return model, None, 'fallback'
    
    raise FileNotFoundError("No models found!")

def initialize_enhanced_system():
    """Initialize the enhanced betting system."""
    try:
        # Load models
        model, selector, model_type = load_optimized_models()
        
        # Load fighter data
        fighters_path = Path('../../model/ufc_fighters_engineered_corrected.csv')
        if not fighters_path.exists():
            fighters_path = Path('../../data/ufc_fighters_engineered_corrected.csv')
        
        if fighters_path.exists():
            fighters_df = pd.read_csv(fighters_path)
            print(f"✅ Loaded {len(fighters_df):,} fighter records")
        else:
            print("⚠️  No fighter data found - predictions will be limited")
            fighters_df = None
        
        # Initialize API client
        print(f"\n🔌 Initializing Odds API client...")
        from ufc_predictor.betting.odds_api_integration import UFCOddsAPIClient
        odds_client = UFCOddsAPIClient(YOUR_ODDS_API_KEY)
        print(f"✅ API client ready")
        
        return {
            'model': model,
            'selector': selector,
            'model_type': model_type,
            'fighters_df': fighters_df,
            'odds_client': odds_client,
            'bankroll': CURRENT_BANKROLL,
            'calibration': USE_CALIBRATION
        }
        
    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        return None

# Initialize the system
print("\n🏗️ Initializing enhanced betting system...")
betting_system = initialize_enhanced_system()

if betting_system:
    print("\n✅ SYSTEM READY!")
    print(f"   • Model: {betting_system['model_type'].upper()}")
    print(f"   • Calibration: {'ENABLED' if betting_system['calibration'] else 'DISABLED'}")
    print(f"   • Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"\n🎮 Ready for enhanced betting analysis!")
else:
    print("\n❌ SYSTEM FAILED TO INITIALIZE")

🎯 ENHANCED UFC BETTING SYSTEM V2.0

💳 Bankroll: $20.00
📊 Max exposure: $4.00
🎯 Min edge: 5.0%

🏗️ Initializing enhanced betting system...

🚀 Loading OPTIMIZED model (32 features)...
✅ Optimized model loaded
   • Accuracy: 73.94%
   • Features: 32
   • Speed: 2.2x faster
✅ Loaded 4,378 fighter records

🔌 Initializing Odds API client...
✅ API client ready

✅ SYSTEM READY!
   • Model: OPTIMIZED
   • Calibration: ENABLED
   • Bankroll: $20.00

🎮 Ready for enhanced betting analysis!


In [7]:
# 📊 CELL 2: AUTO-DETECT UFC EVENTS & FETCH LIVE ODDS
# ====================================================
# Automatically detects upcoming UFC events - no manual input needed!

from ufc_predictor.scrapers.event_discovery import UFCEventDiscovery

def auto_detect_ufc_event_and_odds():
    """Automatically detect next UFC event and fetch odds - zero manual input."""
    print("🔍 AUTO-DETECTING NEXT UFC EVENT")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized. Run Cell 1 first.")
    
    # Initialize event discovery
    discovery = UFCEventDiscovery(api_key=YOUR_ODDS_API_KEY)
    
    # Get next event automatically
    print("🔄 Fetching upcoming UFC events...")
    next_event = discovery.get_next_event(force_refresh=True)
    
    if not next_event:
        print("❌ No upcoming UFC events found")
        print("💡 Possible reasons:")
        print("   • No UFC events scheduled in near future")
        print("   • API rate limit reached")
        print("   • Network connectivity issues")
        return None, None
    
    # Display detected event
    print(f"\n✅ DETECTED UFC EVENT:")
    print(f"   📅 Event: {next_event['name']}")
    print(f"   📆 Date: {next_event['date'].strftime('%Y-%m-%d %H:%M')} UTC")
    print(f"   🥊 Total Fights: {next_event['fight_count']}")
    print(f"   📊 Odds Available: {'Yes' if next_event['odds_available'] else 'No'}")
    
    # Show fight card
    print(f"\n📋 FIGHT CARD:")
    for i, fight in enumerate(next_event['fights'][:12], 1):  # Show up to 12 fights
        print(f"   {i:2}. {fight}")
    
    if len(next_event['fights']) > 12:
        print(f"   ... and {len(next_event['fights']) - 12} more fights")
    
    # Extract odds if available
    if next_event.get('odds_available'):
        print(f"\n💰 EXTRACTING ODDS...")
        odds_data = discovery.extract_odds_for_event(next_event)
        
        if odds_data:
            print(f"✅ Found odds for {len(odds_data)} fights")
            
            # Display odds summary
            print(f"\n📊 ODDS SUMMARY:")
            for i, (fight_key, odds) in enumerate(list(odds_data.items())[:6], 1):
                fighter_a = odds['fighter_a']
                fighter_b = odds['fighter_b']
                odds_a = odds['fighter_a_decimal_odds']
                odds_b = odds['fighter_b_decimal_odds']
                
                # Determine favorite
                if odds_a < odds_b:
                    print(f"   {i}. {fighter_a} ({odds_a:.2f}) vs {fighter_b} ({odds_b:.2f}) ⭐")
                else:
                    print(f"   {i}. {fighter_a} ({odds_a:.2f}) vs {fighter_b} ({odds_b:.2f}) 🎲")
            
            if len(odds_data) > 6:
                print(f"   ... and {len(odds_data) - 6} more fights with odds")
            
            return odds_data, next_event
        else:
            print("⚠️  No valid odds extracted")
            return {}, next_event
    else:
        print("⚠️  No odds available for this event yet")
        return {}, next_event

def check_alternative_events():
    """Check for alternative upcoming UFC events."""
    print("\n📅 CHECKING ALTERNATIVE UFC EVENTS")
    print("=" * 50)
    
    discovery = UFCEventDiscovery(api_key=YOUR_ODDS_API_KEY)
    events = discovery.get_upcoming_events(days_ahead=60)
    
    if not events:
        print("❌ No upcoming events found")
        return None
    
    print(f"Found {len(events)} upcoming UFC events:\n")
    
    for i, event in enumerate(events[:5], 1):
        print(f"{i}. {event['name']}")
        print(f"   Date: {event['date_str']}")
        print(f"   Fights: {event['fight_count']}")
        print(f"   Odds: {'Available' if event['odds_available'] else 'Not yet'}")
        print()
    
    # Allow selection
    choice = input("Select event number (or Enter for first): ").strip()
    
    if choice.isdigit() and 1 <= int(choice) <= min(5, len(events)):
        selected = events[int(choice) - 1]
        discovery = UFCEventDiscovery(api_key=YOUR_ODDS_API_KEY)
        odds_data = discovery.extract_odds_for_event(selected)
        return odds_data, selected
    elif not choice:
        selected = events[0]
        discovery = UFCEventDiscovery(api_key=YOUR_ODDS_API_KEY)
        odds_data = discovery.extract_odds_for_event(selected)
        return odds_data, selected
    else:
        print("❌ Invalid selection")
        return None, None

# ================== MAIN EXECUTION ==================
print("🎯 UFC EVENT AUTO-DETECTION SYSTEM V2.0")
print("=" * 50)
print("No manual input needed - automatically detects next UFC event!\n")

# Automatically detect and fetch odds
odds_data, event_info = auto_detect_ufc_event_and_odds()

if odds_data and event_info:
    # Set global variables for other cells
    EVENT_NAME = event_info['name']
    TARGET_FIGHTS = event_info['fights']
    
    print(f"\n✅ READY FOR BETTING ANALYSIS!")
    print(f"   • Event: {EVENT_NAME}")
    print(f"   • Date: {event_info['date_str']}")
    print(f"   • Fights with odds: {len(odds_data)}")
    print(f"   • Total fights: {len(TARGET_FIGHTS)}")
    
elif event_info:
    # Event found but no odds
    EVENT_NAME = event_info['name']
    TARGET_FIGHTS = event_info['fights']
    
    print(f"\n⚠️  Event detected but no odds available yet")
    print(f"   • Event: {EVENT_NAME}")
    print(f"   • Date: {event_info['date_str']}")
    print(f"   • Check back closer to event date")
    
else:
    print(f"\n❌ Unable to detect UFC events")
    print(f"\n💡 Would you like to check alternative events?")
    
    if input("Check alternatives? (y/n): ").lower() == 'y':
        odds_data, event_info = check_alternative_events()
        if odds_data and event_info:
            EVENT_NAME = event_info['name']
            TARGET_FIGHTS = event_info['fights']
            print(f"\n✅ Alternative event selected: {EVENT_NAME}")

🎯 UFC EVENT AUTO-DETECTION SYSTEM V2.0
No manual input needed - automatically detects next UFC event!

🔍 AUTO-DETECTING NEXT UFC EVENT
🔄 Fetching upcoming UFC events...

✅ DETECTED UFC EVENT:
   📅 Event: UFC_August_09_2025
   📆 Date: 2025-08-09 16:00 UTC
   🥊 Total Fights: 35
   📊 Odds Available: Yes

📋 FIGHT CARD:
    1. Elbert Lukas Steyn vs Abderrahman Errachidy
    2. Abdoul Razac Sankara vs Shadrick Dju Yemba
    3. Desmond Awa Tamungang vs Kunle Lawal
    4. Wilker Nsamo vs Dwight Joseph
    5. Osvaldo Benedito vs Emilios Dassi
    6. Wasiu Adeshina vs Jean-Jacques Lubaya
    7. Michaela Hlavacikova vs Niamh Kinehan
    8. David Zawada vs Kamil Oniszczuk
    9. Piotr Wawrzyniak vs Hojat Khajevand
   10. Krzysztof Jotko vs Marek Mazuch
   11. Raphael Federico vs Vojtěch Kohl
   12. Radek Roušal vs Adrian Hamerski
   ... and 23 more fights

💰 EXTRACTING ODDS...
✅ Found odds for 35 fights

📊 ODDS SUMMARY:
   1. Elbert Lukas Steyn (1.37) vs Abderrahman Errachidy (3.20) ⭐
   2. Abdoul

In [8]:
# 🧠 CELL 3: CALIBRATED PREDICTIONS WITH CONFIDENCE
# ==================================================
# Generate predictions with probability calibration

def apply_calibration(raw_prob: float, strength: float = 0.85) -> float:
    """Apply probability calibration to reduce overconfidence."""
    # Push probabilities toward 0.5 (reduce overconfidence)
    return 0.5 + (raw_prob - 0.5) * strength

def generate_calibrated_predictions():
    """Generate predictions with calibration and enhanced analysis."""
    print("🧠 CALIBRATED PREDICTION ENGINE")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized")
    
    if not odds_data:
        raise Exception("❌ No odds data available")
    
    predictions = []
    
    for fight_key, fight_odds in odds_data.items():
        fighter_a = fight_odds['fighter_a']
        fighter_b = fight_odds['fighter_b']
        
        print(f"\n🥊 {fight_key}")
        print(f"   Analyzing: {fighter_a} vs {fighter_b}")
        
        try:
            # Generate prediction (simplified for demo)
            # In production, this would use full fighter stats
            if betting_system['fighters_df'] is not None:
                # Use actual prediction function
                from ufc_predictor.core.prediction import predict_fight_symmetrical
                
                # Load model columns
                winner_cols = method_cols = None
                try:
                    with open('../../model/winner_model_columns.json', 'r') as f:
                        winner_cols = json.load(f)
                    with open('../../model/method_model_columns.json', 'r') as f:
                        method_cols = json.load(f)
                except:
                    pass
                
                if winner_cols:
                    pred_result = predict_fight_symmetrical(
                        fighter_a, fighter_b,
                        betting_system['fighters_df'],
                        winner_cols, method_cols,
                        betting_system['model'], betting_system['model']
                    )
                    
                    if 'error' not in pred_result:
                        raw_prob_a = float(pred_result['win_probabilities'][fighter_a].replace('%', '')) / 100
                    else:
                        # Fallback to simple prediction
                        raw_prob_a = 0.5
                else:
                    raw_prob_a = 0.5
            else:
                # Demo prediction based on odds
                market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
                # Add some noise to market probability
                raw_prob_a = market_prob_a + np.random.randn() * 0.1
                raw_prob_a = np.clip(raw_prob_a, 0.2, 0.8)
            
            # Apply calibration if enabled
            if betting_system['calibration']:
                calibrated_prob_a = apply_calibration(raw_prob_a, CALIBRATION_STRENGTH)
                print(f"   📊 Raw probability: {raw_prob_a:.1%}")
                print(f"   🎯 Calibrated: {calibrated_prob_a:.1%}")
            else:
                calibrated_prob_a = raw_prob_a
                print(f"   📊 Probability: {calibrated_prob_a:.1%}")
            
            calibrated_prob_b = 1 - calibrated_prob_a
            
            # Market probabilities
            market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
            market_prob_b = 1 / fight_odds['fighter_b_decimal_odds']
            
            # Determine favorites
            model_favorite = fighter_a if calibrated_prob_a > 0.5 else fighter_b
            market_favorite = fighter_a if market_prob_a > market_prob_b else fighter_b
            
            # Calculate expected values
            ev_a = (calibrated_prob_a * fight_odds['fighter_a_decimal_odds']) - 1
            ev_b = (calibrated_prob_b * fight_odds['fighter_b_decimal_odds']) - 1
            
            # Confidence score
            confidence = abs(calibrated_prob_a - 0.5) * 2
            
            # Value assessment
            has_value = max(ev_a, ev_b) > MIN_EDGE_FOR_BET
            is_upset = model_favorite != market_favorite and confidence > 0.3
            
            prediction = {
                'fight': fight_key,
                'fighter_a': fighter_a,
                'fighter_b': fighter_b,
                'raw_prob_a': raw_prob_a,
                'calibrated_prob_a': calibrated_prob_a,
                'calibrated_prob_b': calibrated_prob_b,
                'market_prob_a': market_prob_a,
                'market_prob_b': market_prob_b,
                'odds_a': fight_odds['fighter_a_decimal_odds'],
                'odds_b': fight_odds['fighter_b_decimal_odds'],
                'ev_a': ev_a,
                'ev_b': ev_b,
                'model_favorite': model_favorite,
                'market_favorite': market_favorite,
                'confidence': confidence,
                'has_value': has_value,
                'is_upset': is_upset,
                'best_bet': fighter_a if ev_a > ev_b else fighter_b,
                'best_ev': max(ev_a, ev_b)
            }
            
            predictions.append(prediction)
            
            # Display results
            print(f"   🏆 Model: {model_favorite} ({max(calibrated_prob_a, calibrated_prob_b):.1%})")
            print(f"   📈 Market: {market_favorite} ({max(market_prob_a, market_prob_b):.1%})")
            print(f"   💰 Best EV: {prediction['best_bet']} ({prediction['best_ev']:.1%})")
            
            if has_value:
                print(f"   ✅ VALUE BET FOUND!")
            if is_upset:
                print(f"   🚨 UPSET OPPORTUNITY!")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
            continue
    
    # Summary
    print(f"\n📊 PREDICTION SUMMARY")
    print("=" * 40)
    
    value_bets = [p for p in predictions if p['has_value']]
    upset_bets = [p for p in predictions if p['is_upset']]
    
    print(f"Total fights analyzed: {len(predictions)}")
    print(f"Value bets found: {len(value_bets)}")
    print(f"Upset opportunities: {len(upset_bets)}")
    
    if value_bets:
        print(f"\n💎 TOP VALUE BETS:")
        sorted_bets = sorted(value_bets, key=lambda x: x['best_ev'], reverse=True)
        for i, bet in enumerate(sorted_bets[:3], 1):
            print(f"   {i}. {bet['best_bet']} ({bet['best_ev']:+.1%} EV)")
    
    return predictions

# Generate predictions
print("🎯 Generating calibrated predictions...")
predictions = generate_calibrated_predictions()

if predictions:
    print(f"\n✅ PREDICTIONS READY!")
else:
    print(f"\n❌ No predictions generated")

🎯 Generating calibrated predictions...
🧠 CALIBRATED PREDICTION ENGINE

🥊 Elbert_Lukas_Steyn_vs_Abderrahman_Errachidy
   Analyzing: Elbert Lukas Steyn vs Abderrahman Errachidy
   📊 Raw probability: 50.0%
   🎯 Calibrated: 50.0%
   🏆 Model: Abderrahman Errachidy (50.0%)
   📈 Market: Elbert Lukas Steyn (73.0%)
   💰 Best EV: Abderrahman Errachidy (60.0%)
   ✅ VALUE BET FOUND!

🥊 Abdoul_Razac_Sankara_vs_Shadrick_Dju_Yemba
   Analyzing: Abdoul Razac Sankara vs Shadrick Dju Yemba
   📊 Raw probability: 50.0%
   🎯 Calibrated: 50.0%
   🏆 Model: Shadrick Dju Yemba (50.0%)
   📈 Market: Shadrick Dju Yemba (66.7%)
   💰 Best EV: Abdoul Razac Sankara (22.5%)
   ✅ VALUE BET FOUND!

🥊 Desmond_Awa_Tamungang_vs_Kunle_Lawal
   Analyzing: Desmond Awa Tamungang vs Kunle Lawal
   📊 Raw probability: 50.0%
   🎯 Calibrated: 50.0%
   🏆 Model: Kunle Lawal (50.0%)
   📈 Market: Desmond Awa Tamungang (53.8%)
   💰 Best EV: Kunle Lawal (10.0%)
   ✅ VALUE BET FOUND!

🥊 Wilker_Nsamo_vs_Dwight_Joseph
   Analyzing: Wilker N

In [9]:
# 💰 CELL 4: ENHANCED BANKROLL MANAGEMENT
# ========================================
# Smart Kelly betting with calibration-aware sizing

def calculate_kelly_bet_enhanced(prob: float, odds: float, 
                                 confidence: float, bankroll: float) -> Dict:
    """Calculate Kelly bet with confidence adjustment."""
    
    # Determine bankroll tier
    if bankroll < 200:
        tier = 'MICRO'
        kelly_fraction = 0.15
        max_bet_pct = 0.02
        min_ev = 0.10
    elif bankroll < 1000:
        tier = 'SMALL'
        kelly_fraction = 0.25
        max_bet_pct = 0.05
        min_ev = 0.07
    else:
        tier = 'STANDARD'
        kelly_fraction = 0.35
        max_bet_pct = 0.075
        min_ev = 0.05
    
    # Calculate EV
    ev = (prob * odds) - 1
    
    if ev < min_ev:
        return {
            'bet_size': 0,
            'reason': f"EV too low ({ev:.1%} < {min_ev:.1%})",
            'tier': tier
        }
    
    # Kelly calculation
    kelly_full = (prob * odds - 1) / (odds - 1)
    
    # Apply fractional Kelly
    kelly_adjusted = kelly_full * kelly_fraction
    
    # Apply confidence adjustment (0.5 to 1.0 multiplier)
    confidence_mult = 0.5 + (confidence * 0.5)
    kelly_adjusted *= confidence_mult
    
    # Calculate bet size
    bet_size = bankroll * kelly_adjusted
    
    # Apply max constraint
    max_bet = bankroll * max_bet_pct
    bet_size = min(bet_size, max_bet)
    
    # Apply minimum
    min_bet = max(5, bankroll * 0.005)
    if bet_size < min_bet:
        if ev > min_ev * 1.5:  # Only bet if EV is really good
            bet_size = min_bet
        else:
            return {
                'bet_size': 0,
                'reason': f"Bet too small (${bet_size:.2f})",
                'tier': tier
            }
    
    return {
        'bet_size': round(bet_size, 2),
        'kelly_full': kelly_full,
        'kelly_fraction': kelly_fraction,
        'confidence_mult': confidence_mult,
        'ev': ev,
        'tier': tier,
        'potential_profit': round(bet_size * (odds - 1), 2)
    }

def generate_betting_portfolio():
    """Generate complete betting portfolio with risk management."""
    print("💰 SMART BETTING PORTFOLIO GENERATION")
    print("=" * 50)
    
    if not predictions:
        raise Exception("❌ No predictions available")
    
    bankroll = betting_system['bankroll']
    
    # Determine strategy tier
    if bankroll < 200:
        strategy = "MICRO - Capital preservation focus"
    elif bankroll < 1000:
        strategy = "SMALL - Conservative growth"
    else:
        strategy = "STANDARD - Moderate Kelly"
    
    print(f"💳 Bankroll: ${bankroll:,.2f}")
    print(f"📊 Strategy: {strategy}")
    print(f"🎯 Max exposure: ${bankroll * MAX_TOTAL_EXPOSURE:.2f}")
    
    recommendations = []
    total_allocated = 0
    max_exposure = bankroll * MAX_TOTAL_EXPOSURE
    
    # Sort predictions by EV
    sorted_preds = sorted(predictions, key=lambda x: x['best_ev'], reverse=True)
    
    print(f"\n🎯 ANALYZING OPPORTUNITIES")
    print("-" * 40)
    
    for pred in sorted_preds:
        # Check both fighters
        for fighter_idx in ['a', 'b']:
            if fighter_idx == 'a':
                fighter = pred['fighter_a']
                prob = pred['calibrated_prob_a']
                odds = pred['odds_a']
                ev = pred['ev_a']
            else:
                fighter = pred['fighter_b']
                prob = pred['calibrated_prob_b']
                odds = pred['odds_b']
                ev = pred['ev_b']
            
            # Skip if no edge
            if ev < MIN_EDGE_FOR_BET:
                continue
            
            # Check remaining exposure
            remaining = max_exposure - total_allocated
            if remaining < bankroll * 0.01:
                continue
            
            # Calculate bet
            bet_calc = calculate_kelly_bet_enhanced(
                prob, odds, pred['confidence'], bankroll
            )
            
            if bet_calc['bet_size'] == 0:
                continue
            
            # Adjust for remaining exposure
            final_bet = min(bet_calc['bet_size'], remaining)
            
            recommendation = {
                'fight': pred['fight'],
                'fighter': fighter,
                'opponent': pred['fighter_b'] if fighter_idx == 'a' else pred['fighter_a'],
                'probability': prob,
                'odds': odds,
                'ev': ev,
                'confidence': pred['confidence'],
                'bet_size': final_bet,
                'potential_profit': final_bet * (odds - 1),
                'kelly_info': bet_calc,
                'is_upset': pred['is_upset'],
                'is_favorite': fighter == pred['market_favorite']
            }
            
            recommendations.append(recommendation)
            total_allocated += final_bet
            
            print(f"\n✅ {fighter}")
            print(f"   Stake: ${final_bet:.2f}")
            print(f"   Odds: {odds:.2f}")
            print(f"   EV: {ev:+.1%}")
            print(f"   Potential: ${final_bet * (odds - 1):.2f}")
            if pred['is_upset']:
                print(f"   🚨 UPSET BET")
    
    # Portfolio summary
    print(f"\n📊 PORTFOLIO SUMMARY")
    print("=" * 40)
    
    if recommendations:
        total_stake = sum(r['bet_size'] for r in recommendations)
        total_potential = sum(r['potential_profit'] for r in recommendations)
        avg_ev = sum(r['ev'] * r['bet_size'] for r in recommendations) / total_stake
        
        print(f"Recommended bets: {len(recommendations)}")
        print(f"Total stake: ${total_stake:.2f}")
        print(f"Bankroll usage: {total_stake/bankroll*100:.1f}%")
        print(f"Expected value: {avg_ev:+.1%}")
        print(f"Potential profit: ${total_potential:.2f}")
        
        # Risk assessment
        upset_bets = [r for r in recommendations if r['is_upset']]
        favorite_bets = [r for r in recommendations if r['is_favorite']]
        
        print(f"\n🎲 RISK PROFILE:")
        print(f"   Favorites: {len(favorite_bets)} bets")
        print(f"   Underdogs: {len(recommendations) - len(favorite_bets)} bets")
        print(f"   Upset plays: {len(upset_bets)} bets")
        
    else:
        print("📭 No betting opportunities found")
        print("Reasons: Insufficient edge or bankroll constraints")
    
    return recommendations

# Generate portfolio
print("🎯 Generating betting portfolio...")
betting_portfolio = generate_betting_portfolio()

if betting_portfolio:
    print(f"\n✅ PORTFOLIO READY!")
else:
    print(f"\n📭 No bets recommended")

🎯 Generating betting portfolio...
💰 SMART BETTING PORTFOLIO GENERATION
💳 Bankroll: $20.00
📊 Strategy: MICRO - Capital preservation focus
🎯 Max exposure: $4.00

🎯 ANALYZING OPPORTUNITIES
----------------------------------------

✅ Toshiomi Kazama
   Stake: $4.00
   Odds: 7.00
   EV: +250.0%
   Potential: $24.00

📊 PORTFOLIO SUMMARY
Recommended bets: 1
Total stake: $4.00
Bankroll usage: 20.0%
Expected value: +250.0%
Potential profit: $24.00

🎲 RISK PROFILE:
   Favorites: 0 bets
   Underdogs: 1 bets
   Upset plays: 0 bets

✅ PORTFOLIO READY!


In [10]:
# 📝 CELL 5: SAVE & TRACK WITH ADVANCED BET TRACKING SYSTEM
# =======================================================
# Professional bet tracking with deduplication and performance analysis

# Import the comprehensive bet tracking system
from ufc_predictor.betting.bet_tracking import BetTracker

def save_betting_portfolio_advanced():
    """Save betting portfolio using the advanced tracking system."""
    print("📝 SAVING BETTING PORTFOLIO (ADVANCED)")
    print("=" * 50)
    
    if not betting_portfolio:
        print("📭 No bets to save")
        return []
    
    # Initialize the bet tracker
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    saved_bets = []
    event_date = datetime.now().strftime('%Y-%m-%d')
    
    print(f"📋 Processing {len(betting_portfolio)} recommendations...")
    print("-" * 40)
    
    for i, bet in enumerate(betting_portfolio, 1):
        # Prepare method prediction
        method_pred = "Decision"  # Default, could be enhanced with actual method predictions
        if bet.get('is_upset'):
            method_pred = "Upset opportunity"
        
        # Log the bet using the comprehensive tracker
        bet_id = tracker.log_bet_recommendation(
            event_name=EVENT_NAME,
            event_date=event_date,
            fighter=bet['fighter'],
            opponent=bet['opponent'],
            bet_type='SINGLE',
            decimal_odds=bet['odds'],
            american_odds=int((bet['odds'] - 1) * 100) if bet['odds'] >= 2 else int(-100/(bet['odds']-1)),
            bet_size=bet['bet_size'],
            expected_value=bet['ev'],
            expected_return=bet['potential_profit'],
            model_probability=bet['probability'],
            market_probability=1 / bet['odds'],
            risk_level=bet['kelly_info']['tier'],
            source=f"Enhanced_{betting_system['model_type'].upper()}",
            method_prediction=method_pred,
            bankroll=betting_system['bankroll']
        )
        
        if bet_id:
            saved_bets.append(bet_id)
            print(f"   {i}. ✅ {bet['fighter']} vs {bet['opponent']}")
        else:
            print(f"   {i}. ⚠️  Duplicate or error: {bet['fighter']}")
    
    print(f"\n📊 Summary:")
    print(f"   • Recommendations: {len(betting_portfolio)}")
    print(f"   • New bets saved: {len(saved_bets)}")
    print(f"   • Duplicates skipped: {len(betting_portfolio) - len(saved_bets)}")
    
    return saved_bets

def check_duplicates_and_clean():
    """Check for and remove duplicate bets in the CSV."""
    print("\n🔍 CHECKING FOR DUPLICATES")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Validate data integrity
    validation = tracker.validate_data()
    print(f"📊 Total records: {validation.get('total_records', 0)}")
    print(f"🔍 Data quality: {validation.get('data_quality', 'UNKNOWN')}")
    
    if validation.get('issues'):
        print(f"\n⚠️  Issues found:")
        for issue in validation['issues']:
            print(f"   • {issue}")
    
    # Remove duplicates
    duplicates_removed = tracker.remove_duplicate_recommendations()
    
    return duplicates_removed

def generate_quick_report():
    """Generate a quick performance report."""
    print("\n📈 PERFORMANCE ANALYSIS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Generate reports for different periods
    print("\n🔹 Last 7 days:")
    report_7d = tracker.generate_performance_report(days=7)
    
    print("\n🔹 Last 30 days:")
    report_30d = tracker.generate_performance_report(days=30)
    
    print("\n🔹 All time:")
    report_all = tracker.generate_performance_report(days=0)
    
    return report_all

def display_final_betting_card():
    """Display professional betting card."""
    print("\n" + "="*70)
    print("FINAL BETTING CARD")
    print("="*70)
    
    if not betting_portfolio:
        print("📭 No bets for this event")
        return
    
    print(f"Event: {EVENT_NAME}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
    print(f"Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"Model: {betting_system['model_type'].upper()}")
    print(f"Calibration: {'ON' if betting_system['calibration'] else 'OFF'}")
    
    total_stake = sum(b['bet_size'] for b in betting_portfolio)
    total_potential = sum(b['potential_profit'] for b in betting_portfolio)
    
    print(f"\nTotal Stake: ${total_stake:.2f}")
    print(f"Potential Return: ${total_potential:.2f}")
    print(f"ROI: {(total_potential/total_stake)*100:.1f}%")
    
    print(f"\n{'Fighter':<25} {'Odds':<8} {'Stake':<10} {'EV':<8} {'Profit':<10}")
    print("-" * 70)
    
    for bet in betting_portfolio:
        mark = "🚨" if bet['is_upset'] else "⭐" if bet['is_favorite'] else "📊"
        print(f"{bet['fighter']:<25} {bet['odds']:<8.2f} "
              f"${bet['bet_size']:<9.2f} {bet['ev']:<7.1%} "
              f"${bet['potential_profit']:<9.2f} {mark}")
    
    print("-" * 70)
    print(f"{'TOTAL':<35} ${total_stake:<9.2f} "
          f"{'':8} ${total_potential:<9.2f}")
    
    print("\n📋 LEGEND:")
    print("   ⭐ = Favorite | 🚨 = Upset | 📊 = Value")

def update_fight_results_interactive():
    """Interactive function to update fight results after the event."""
    print("\n🥊 UPDATE FIGHT RESULTS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Get pending bets for the current event
    pending = tracker.get_bets_by_event(EVENT_NAME)
    pending = pending[pending['actual_result'].isna()]
    
    if pending.empty:
        print("📭 No pending bets for this event")
        return
    
    print(f"Found {len(pending)} pending bets for {EVENT_NAME}")
    print("\nUpdate each result (WIN/LOSS/PUSH):")
    
    for _, bet in pending.iterrows():
        print(f"\n{bet['fighter']} vs {bet['opponent']}")
        print(f"   Stake: ${bet['bet_size']:.2f} @ {bet['odds_decimal']:.2f}")
        
        result = input("   Result (W/L/P or skip): ").upper()
        
        if result in ['W', 'WIN']:
            profit = bet['bet_size'] * (bet['odds_decimal'] - 1)
            tracker.update_fight_result(bet['bet_id'], 'WIN', profit)
        elif result in ['L', 'LOSS']:
            tracker.update_fight_result(bet['bet_id'], 'LOSS', -bet['bet_size'])
        elif result in ['P', 'PUSH']:
            tracker.update_fight_result(bet['bet_id'], 'PUSH', 0)
        else:
            print("   Skipped")

# ================== MAIN EXECUTION ==================
print("💾 Ready to save betting portfolio...\n")

# Display final card first
display_final_betting_card()

# Ask for confirmation
if betting_portfolio:
    print("\n🎯 SAVE OPTIONS:")
    print("1. Save with duplicate detection (recommended)")
    print("2. Clean duplicates from existing records")
    print("3. Generate performance report")
    print("4. Skip saving")
    
    choice = input("\nSelect option (1-4): ")
    
    if choice == '1':
        saved_records = save_betting_portfolio_advanced()
        print("\n✅ BETTING ANALYSIS COMPLETE!")
        print("🎯 Good luck with your bets!")
        print("📝 Use update_fight_results_interactive() after the fights")
        
    elif choice == '2':
        duplicates_removed = check_duplicates_and_clean()
        print(f"\n✅ Cleaned {duplicates_removed} duplicate records")
        
    elif choice == '3':
        report = generate_quick_report()
        
    else:
        print("\n❌ Bets not saved")
else:
    print("\n✅ Analysis complete - no bets recommended")

💾 Ready to save betting portfolio...


FINAL BETTING CARD
Event: UFC_August_09_2025
Date: 2025-08-10
Bankroll: $20.00
Model: OPTIMIZED
Calibration: ON

Total Stake: $4.00
Potential Return: $24.00
ROI: 600.0%

Fighter                   Odds     Stake      EV       Profit    
----------------------------------------------------------------------
Toshiomi Kazama           7.00     $4.00      250.0%  $24.00     📊
----------------------------------------------------------------------
TOTAL                               $4.00               $24.00    

📋 LEGEND:
   ⭐ = Favorite | 🚨 = Upset | 📊 = Value

🎯 SAVE OPTIONS:
1. Save with duplicate detection (recommended)
2. Clean duplicates from existing records
3. Generate performance report
4. Skip saving
📝 SAVING BETTING PORTFOLIO (ADVANCED)
📊 Using existing betting records: /Users/diyagamah/Documents/ufc-predictor/betting_records.csv
📋 Processing 1 recommendations...
----------------------------------------
💾 Backup created: /Users/diyagamah/Doc

In [ ]:
# 🏆 CELL 6: UPDATE FIGHT RESULTS (Run After Event)
# ==================================================
# Run this cell after the UFC event to update your betting results

# Update fight results interactively
update_fight_results_interactive()

# Generate performance report after updating
print("\n📊 Would you like to see your performance report? (y/n)")
if input("> ").lower() == 'y':
    generate_quick_report()

print("\n🎉 Thank you for using the Enhanced UFC Betting System!")
print("💪 Good luck with your next event!")